# Diagnosis of cancer

In this post I will describe how to build a deep neural network that can predict whether a tumor is malignant or benign. In the earlier [post](https://nathdip.github.io/cancer%20diagnosis%20I.html), we have looked into how we can use a linear classifier and just two features to classify the different tumors. However with the large number of features at our disposal a logical way to get the best classification is to train a neural network. 

## Designing a deep neural network.

Neural networks consists of network of interconnected nodes or neurons as it is inspired by connection among neurons in the brain. The various features form the input layer of neuron. There is also another set of neuron(s) which form the output layer. Connecting the input layer and the output layer are the hidden layer. Depending upon the application we might have several such hidden layers. Deep neural networks consists of several such hidden layers. 

In this particular case we will have a neural network which has an input layer of size number of features. Let us have a look at our data again.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('data.csv')
dataset.head(n=10)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,NaN
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,NaN
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,NaN


We need to drop the last column of the dataframe as it has only 'NaN' as entries

In [4]:
dataset = dataset.drop(columns = 'Unnamed: 32', axis = 1)

In [5]:
dataset.head(n=5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Now let us look at what will constitute as features which will be the input layer of our neural network. Clearly the id of the patient should play no role in the diagnosis and we can ignore it for the rest of the discussion. The column diagnosis corresponds to the output of the the neural network which we will use for the purpose of training our neural network. We can now split our data into input (X) and output (y) components.

In [17]:
features = list(dataset)
features = np.delete(features, [0,1])
X = dataset.loc[:, features].values
y = dataset.loc[:, 'diagnosis'].values

Now that we have the dataset a little preprocessing needs to be done in order to able to train the neural network. First we need to feature scale all the inputs. We will use a standard scaler in this case which basically converts all the inputs which has a zero mean and a unit standard deviation.

In [20]:
from sklearn.preprocessing import StandardScaler
sc =  StandardScaler()
X = sc.fit_transform(X)

The output vector y which is the diagnosis has two classes 'M' for malignant and 'B' for benign. We need to encode it into a numeric values such that our classifier can understand

In [22]:
from sklearn.preprocessing import LabelEncoder
labelencode = LabelEncoder()
y = labelencode.fit_transform(y)

Now that we have our data processed and ready, we are ready to build our classifier. We will be using Keras which is an API for packages for advanced machine learning like TensorFlow, Theano etc. We will be using the Keras API with the TensorFlow backend. As a first trial we will split the X and y data set into training and test set so that evaluate the performance of our neural network. 

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


Now we will import all the libraries required to build our neural network.

In [44]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
X_train.shape

(455, 30)

Using Sequential and Dense we will build our classifier and we will be able to add layers to our network. 

In [52]:
classifier = Sequential()

#Add the input layer and first hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30)  )
classifier.add(Dropout(rate=0.1))
#Add second hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add third hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add fourth hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add fifth hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add sixth hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add seventh hidden layer
classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu')  )
classifier.add(Dropout(rate=0.1))

#Add eighth hidden layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')  )

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Epoch 1/100
455/455 [==============================] - 0s - loss: 0.6902 - acc: 0.6374     
Epoch 2/100
455/455 [==============================] - 0s - loss: 0.6769 - acc: 0.6374     
Epoch 3/100
455/455 [==============================] - 0s - loss: 0.4298 - acc: 0.6374     
Epoch 4/100
455/455 [==============================] - 0s - loss: 0.3300 - acc: 0.6615     
Epoch 5/100
455/455 [==============================] - 0s - loss: 0.2823 - acc: 0.9758     
Epoch 6/100
455/455 [==============================] - 0s - loss: 0.2694 - acc: 0.9846     
Epoch 7/100
455/455 [==============================] - 0s - loss: 0.2040 - acc: 0.9890     
Epoch 8/100
455/455 [==============================] - 0s - loss: 0.1185 - acc: 0.9802     
Epoch 9/100
455/455 [==============================] - 0s - loss: 0.0781 - acc: 0.9890     
Epoch 10/100
455/455 [==============================] - 0s - loss: 0.0689 - acc: 0.9868     
Epoch 11/100
455/455 [==============================] - 0s - loss: 0.0767 - acc

455/455 [==============================] - 0s - loss: 0.0131 - acc: 0.9956        
Epoch 89/100
455/455 [==============================] - 0s - loss: 0.0081 - acc: 0.9978     
Epoch 90/100
455/455 [==============================] - 0s - loss: 0.0290 - acc: 0.9934     
Epoch 91/100
455/455 [==============================] - 0s - loss: 0.0085 - acc: 0.9978     
Epoch 92/100
455/455 [==============================] - 0s - loss: 0.0094 - acc: 0.9978     
Epoch 93/100
455/455 [==============================] - 0s - loss: 0.0060 - acc: 0.9978        
Epoch 94/100
455/455 [==============================] - 0s - loss: 0.0125 - acc: 0.9978        
Epoch 95/100
455/455 [==============================] - 0s - loss: 0.0118 - acc: 0.9956        
Epoch 96/100
455/455 [==============================] - 0s - loss: 0.0128 - acc: 0.9956     
Epoch 97/100
455/455 [==============================] - 0s - loss: 0.0123 - acc: 0.9956     
Epoch 98/100
455/455 [==============================] - 0s - loss: 0.00

Our classifier seems to have a really good accuracy on the training set. But that might be just because of overfitting. The Dropout is used as a form of regularization while training our classifier. However in order to know the actual accuracy of our classifier we need to implement on the test set and how well our classifier performs.

In [59]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [60]:
accuracy_cm = (cm[0, 0] + cm[1, 1])/(sum(sum(cm)))
print('The accuracy is : ', accuracy_cm)


The accuracy is :  0.964912280702


This is not bad, our algorithm is able to distinguish between malignant and benign tumors with an accuracy of nearly 96.5%. Remember this dataset is not large but with the extensive number of features and a deep artificial neural network, we are able to make a very good prediction. We still have nearly 3.5% chance of getting it wrong. There also seems to be a difference between the accuracy on the training set and that on the test set. We would also like to close that divide if possible.